## Processing of other validation dataset

### Load predictions and validation data

In [262]:
import json
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
from nltk import tokenize
import spacy
import string
import re

[nltk_data] Downloading package stopwords to /home/luki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/luki/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [328]:
# validation_dataset = 'nq_dev_formated.json'
# validation_dataset = 'adversarialQA_validation.json'
# validation_dataset = 'valid_pred_labeled.json'
# file_folder = './'
file_folder = './from_debiased_models/'
# validation_dataset = 'squad_supersampled_similar_words_4.json'
# validation_dataset = 'squad_supersampled_answer_length_3.json'
# validation_dataset = 'squad_supersampled_distances_7.json'

# validation_dataset = 'squad_supersampled_all_v1.json'
# validation_dataset = 'squad_supersampled_all_v2.json'
# validation_dataset = 'squad_supersampled_all_dd_ns.json'
# validation_dataset = 'squad_supersampled_ans_sub_pos_1.json'
# validation_dataset = 'squad_supersampled_cosine_similarity_01.json'
validation_dataset = 'squad_supersampled_similar_entities_0.json'

### Count similar words between question and context

In [329]:
data = pd.read_json(file_folder + validation_dataset)

In [330]:
data = data.reset_index()
data

,index,id,title,context,question,answers,prediction_text
0,0,56be4db0acb8001400a502ec,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,"{'text': ['Denver Broncos', 'Denver Broncos', ...",Denver Broncos
1,1,56be4db0acb8001400a502ed,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,"{'text': ['Carolina Panthers', 'Carolina Panth...",Carolina Panthers
2,2,56be4db0acb8001400a502ee,Super_Bowl_50,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,"{'text': ['Santa Clara, California', 'Levi's S...","Santa Clara, California"
3,3,56be4db0acb8001400a502ef,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team won Super Bowl 50?,"{'text': ['Denver Broncos', 'Denver Broncos', ...",Denver Broncos
4,4,56be4db0acb8001400a502f0,Super_Bowl_50,Super Bowl 50 was an American football game to...,What color was used to emphasize the 50th anni...,"{'text': ['gold', 'gold', 'gold'], 'answer_sta...",gold
...,...,...,...,...,...,...,...
10565,10565,5737aafd1c456719005744fb,Force,"The pound-force has a metric counterpart, less...",What is the metric term less used than the New...,"{'text': ['kilogram-force', 'pound-force', 'ki...",pound-force
10566,10566,5737aafd1c456719005744fc,Force,"The pound-force has a metric counterpart, less...",What is the kilogram-force sometimes reffered ...,"{'text': ['kilopond', 'kilopond', 'kilopond', ...",kilopond
10567,10567,5737aafd1c456719005744fd,Force,"The pound-force has a metric counterpart, less...",What is a very seldom used unit of mass in the...,"{'text': ['slug', 'metric slug', 'metric slug'...",the metric slug
10568,10568,5737aafd1c456719005744fe,Force,"The pound-force has a metric counterpart, less...",What seldom used term of a unit of force equal...,"{'text': ['kip', 'kip', 'kip', 'kip', 'kip'], ...","alternate, but rarely used unit of mass: the m..."


In [331]:
def count_similar_words_in_question_and_context(data):

    tokenizer = nltk.RegexpTokenizer(r"\w+")

    similar_words = []

    for i in range(len(data)):
        context1 = nltk.word_tokenize(data['context'][i])
        question1 = nltk.word_tokenize(data['question'][i])
        context_new = [word for word in context1 if word.isalnum()]
        question_new = [word for word in question1 if word.isalnum()]
        similar_words.append(len(set(context_new).intersection(set(question_new))))
        
    return similar_words

In [332]:
data['similar_words'] = count_similar_words_in_question_and_context(data)
data

,index,id,title,context,question,answers,prediction_text,similar_words
0,0,56be4db0acb8001400a502ec,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,"{'text': ['Denver Broncos', 'Denver Broncos', ...",Denver Broncos,7
1,1,56be4db0acb8001400a502ed,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,"{'text': ['Carolina Panthers', 'Carolina Panth...",Carolina Panthers,7
2,2,56be4db0acb8001400a502ee,Super_Bowl_50,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,"{'text': ['Santa Clara, California', 'Levi's S...","Santa Clara, California",3
3,3,56be4db0acb8001400a502ef,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team won Super Bowl 50?,"{'text': ['Denver Broncos', 'Denver Broncos', ...",Denver Broncos,4
4,4,56be4db0acb8001400a502f0,Super_Bowl_50,Super Bowl 50 was an American football game to...,What color was used to emphasize the 50th anni...,"{'text': ['gold', 'gold', 'gold'], 'answer_sta...",gold,8
...,...,...,...,...,...,...,...,...
10565,10565,5737aafd1c456719005744fb,Force,"The pound-force has a metric counterpart, less...",What is the metric term less used than the New...,"{'text': ['kilogram-force', 'pound-force', 'ki...",pound-force,6
10566,10566,5737aafd1c456719005744fc,Force,"The pound-force has a metric counterpart, less...",What is the kilogram-force sometimes reffered ...,"{'text': ['kilopond', 'kilopond', 'kilopond', ...",kilopond,5
10567,10567,5737aafd1c456719005744fd,Force,"The pound-force has a metric counterpart, less...",What is a very seldom used unit of mass in the...,"{'text': ['slug', 'metric slug', 'metric slug'...",the metric slug,9
10568,10568,5737aafd1c456719005744fe,Force,"The pound-force has a metric counterpart, less...",What seldom used term of a unit of force equal...,"{'text': ['kip', 'kip', 'kip', 'kip', 'kip'], ...","alternate, but rarely used unit of mass: the m...",7


In [333]:
data['similar_words'].describe()

count    10570.000000
mean         6.047020
std          2.813025
min          0.000000
25%          4.000000
50%          6.000000
75%          8.000000
max         28.000000
Name: similar_words, dtype: float64

In [334]:
data['similar_words'].value_counts()

5     1635
6     1529
4     1387
7     1295
3     1084
8      998
9      699
2      554
10     431
11     280
1      222
12     163
13     110
14      61
0       34
15      29
16      26
17      14
19       6
18       5
20       3
21       2
28       1
22       1
23       1
Name: similar_words, dtype: int64

In [335]:
# data.to_json('enhanced_'+validation_dataset, orient='records')

### Count distance of the closest question word from answer in context

In [336]:
# data = pd.read_json('enhanced_'+validation_dataset)

In [337]:
# code from this web site https://www.codegrepper.com/code-examples/python/find+index+of+sublist+in+list+python
def find_sub_list(sl,l):
    results=[]
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            results.append((ind,ind+sll-1))

    return results

In [338]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

def count_lowest_position_of_word_from_question_in_context(data):
    tokenizer = RegexpTokenizer(r'\w+')
    distances = []
    words = []

    for i in range(len(data)):        
        indexes_of_words = []
        context_list = tokenizer.tokenize(data['context'][i])
        question_list = tokenizer.tokenize(data['question'][i])
        answer_text = tokenizer.tokenize(data['answers'][i]['text'][0])
        answer_start = data['answers'][i]['answer_start']

        indexes_of_words = find_sub_list(answer_text, context_list)

        if len(indexes_of_words) > 0:
            answer_index = indexes_of_words[0][0]
        else:
            distances.append(-1)
            words.append('None')
            continue

        filtered_words = [word for word in question_list if word not in stopwords.words('english')]

        list_indexes = {}

        for word in filtered_words:
            if word in context_list:
                for j in range(len(context_list)):
                    if word == context_list[j]:
                        list_indexes[abs(j - answer_index)] = context_list[j]

        sort_orders = sorted(list_indexes.items(), key=lambda x: x[0], reverse=False)

        if len(sort_orders) == 0:
            distances.append(-1)
            words.append('None')
        else:
            distances.append(sort_orders[0][0])
            words.append(sort_orders[0][1])

    return distances, words

[nltk_data] Downloading package stopwords to /home/luki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [339]:
data['str_answers'] = data['answers'].astype('str')

In [340]:
data['distances'], data['closest_words'] = count_lowest_position_of_word_from_question_in_context(data)

In [341]:
data['distances'].value_counts()

2      2478
1      2243
3      1404
4       886
5       617
       ... 
84        1
108       1
276       1
85        1
127       1
Name: distances, Length: 98, dtype: int64

In [342]:
# data.to_json('enhanced_'+validation_dataset, orient='records')

### Identify in which sentence the answer is

In [343]:
# data = pd.read_json('enhanced_'+validation_dataset)

In [344]:
from nltk import tokenize

def identify_in_which_sentence_answer_is(data):
    sentence_indexes = []

    for i in range(len(data)):
        context1 = tokenize.sent_tokenize(data['context'][i])
        answer = data['answers'][i]['text'][0]
        nth = 0
        for sentence in context1:
            if answer in sentence:
                break
            nth += 1

        sentence_indexes.append(nth)

    return sentence_indexes

In [345]:
data['kth_sentence'] = identify_in_which_sentence_answer_is(data)

In [346]:
data['kth_sentence'].describe()

count    10570.000000
mean         1.643992
std          1.884932
min          0.000000
25%          0.000000
50%          1.000000
75%          3.000000
max         29.000000
Name: kth_sentence, dtype: float64

In [347]:
data['kth_sentence'].describe()

count    10570.000000
mean         1.643992
std          1.884932
min          0.000000
25%          0.000000
50%          1.000000
75%          3.000000
max         29.000000
Name: kth_sentence, dtype: float64

In [348]:
data['kth_sentence'].value_counts()

0     3571
1     2511
2     1790
3     1262
4      679
5      336
6      191
7      109
8       65
9       21
10      10
11       5
13       4
15       4
12       4
29       2
14       2
16       2
26       1
27       1
Name: kth_sentence, dtype: int64

In [349]:
data['kth_sentence'].value_counts()

0     3571
1     2511
2     1790
3     1262
4      679
5      336
6      191
7      109
8       65
9       21
10      10
11       5
13       4
15       4
12       4
29       2
14       2
16       2
26       1
27       1
Name: kth_sentence, dtype: int64

In [350]:
# data.to_json('enhanced_'+validation_dataset, orient='records')

### Computing cosine similarity from tf-idf between contexts and questions

In [351]:
# data = pd.read_json('enhanced_'+validation_dataset)

In [352]:
train_dataset = pd.read_json('./train.json')

In [353]:
train_dataset

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...
...,...,...,...,...,...
87594,5735d259012e2f140011a09d,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,"{'text': ['Oregon'], 'answer_start': [229]}"
87595,5735d259012e2f140011a09e,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,"{'text': ['Rangoon'], 'answer_start': [414]}"
87596,5735d259012e2f140011a09f,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,"{'text': ['Minsk'], 'answer_start': [476]}"
87597,5735d259012e2f140011a0a0,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,"{'text': ['1975'], 'answer_start': [199]}"


In [354]:
vectorizer = TfidfVectorizer(stop_words='english', use_idf=True)
model = vectorizer.fit(train_dataset['context']) # fit na vsetkych datach

def compute_similarity_between_context_and_question(data):
    similarities = []

    for i in tqdm(range(len(data))):
        context1 = vectorizer.transform([data['context'][i]])
        question1 = vectorizer.transform([data['question'][i]])
        similarities.append(cosine_similarity(context1, question1)[0][0])

    return similarities


In [355]:
data['cosine_similarity'] = compute_similarity_between_context_and_question(data)

  0%|          | 0/10570 [00:00<?, ?it/s]

In [356]:
data['cosine_similarity'].describe()

count    10570.000000
mean         0.306264
std          0.158483
min          0.000000
25%          0.187675
50%          0.296845
75%          0.418278
max          0.910359
Name: cosine_similarity, dtype: float64

In [357]:
data['cosine_similarity'].describe()

count    10570.000000
mean         0.306264
std          0.158483
min          0.000000
25%          0.187675
50%          0.296845
75%          0.418278
max          0.910359
Name: cosine_similarity, dtype: float64

In [358]:
# data.to_json('enhanced_'+validation_dataset, orient='records')

In [359]:
data_distances = data[data.distances >= 0]
data_higher, data_lower = [x for _, x in data.groupby(data_distances['distances'] <= 3)]

### Extract length of the answer

In [360]:
# data = pd.read_json('enhanced_'+validation_dataset)

In [361]:
def average_answer_length(data):
    answers_text = []
    for i in range(len(data)):
        answers_text.append(data['answers'][i]['text'])
    answers_text

    answer_lenght = []
    tokenizer = RegexpTokenizer(r'\w+')
    for i in range(len(data)):
        avg_lenght = 0
        for j in range(len(answers_text[i])):
            avg_lenght += len(tokenizer.tokenize(answers_text[i][j]))
        av = avg_lenght/(len(answers_text[i]))
        answer_lenght.append(av)

    return answer_lenght

In [362]:
data['answer_lenght'] = average_answer_length(data)

In [363]:
data['answer_lenght'].describe()

count    10570.000000
mean         3.008505
std          2.457632
min          0.800000
25%          1.333333
50%          2.000000
75%          3.666667
max         24.333333
Name: answer_lenght, dtype: float64

In [364]:
data['answer_lenght'].value_counts()

1.000000     2294
2.000000     1842
3.000000     1016
1.666667      516
2.333333      470
             ... 
14.800000       1
18.666667       1
22.333333       1
17.666667       1
13.600000       1
Name: answer_lenght, Length: 164, dtype: int64

In [365]:
# data.to_json('enhanced_'+validation_dataset, orient='records')

### Extract number of similar NER to answer from context

In [366]:
# data = pd.read_json('enhanced_'+validation_dataset)

In [367]:
nlp = spacy.load('en_core_web_sm')

In [368]:
def show_ents(doc): 
    entities = []
    if doc.ents: 
        for ent in doc.ents: 
            entities.append(ent.label_)
    return entities

In [369]:
def count_similar_NER_from_context_to_answer(data):
    context_ents = []
    answer_ents = []

    for row in range(len(data)):
        context = nlp(data['context'][row])
        context_ents.append(show_ents(context))
        for ans in data['answers'][row]['text']:
            act_ans_ents_ = []
            act_ans = nlp(ans)
            act_ans_ents_.append(show_ents(act_ans))
        answer_ents.append(act_ans_ents_)

    max_sim_ents = []

    for row, cont in zip(answer_ents, context_ents):
        max = 0
        for items in row:
            for item in items:
                if cont.count(item) > max:
                    max = cont.count(item)
        max_sim_ents.append(max)
    
    return max_sim_ents

In [370]:
data['max_sim_ents'] = count_similar_NER_from_context_to_answer(data)

In [371]:
data['max_sim_ents'].value_counts()

0     5006
1     1015
2     1005
3      817
4      620
5      531
6      370
7      299
8      241
9      148
10      80
12      69
15      69
11      67
13      52
19      34
17      26
16      22
14      17
22      14
20      12
26      11
31      10
18       8
33       6
25       6
44       4
21       3
23       3
27       3
29       2
Name: max_sim_ents, dtype: int64

In [372]:
# data.to_json('enhanced_'+validation_dataset, orient='records')

### Extract number of occurencies of subject from question in context

In [373]:

# data = pd.read_json('enhanced_'+validation_dataset)

In [374]:
nlp = spacy.load('en_core_web_sm')

In [375]:
def doc_pieces(doc):
    subjects = []
    for ent in doc:
        if ent.dep_ == 'nsubj':
            subjects.append(ent.text)
    return subjects

In [376]:
q_subjects = []

for item in data['question']:
    question = nlp(item)
    q_subjects.append(doc_pieces(question))
    

In [377]:
subject_in_context_count = []

for context, q_sub in zip(data['context'], q_subjects):
    context_with_no_punct = context.translate(str.maketrans('', '', string.punctuation))
    # print(context_with_no_punct)
    count = 0
    max = 0
    for item in q_sub:
        if item in context_with_no_punct:
            # print(f'Index {context_with_no_punct.index(item)}\nWord {item}\nCotext {context_with_no_punct}')
            
            indexes = [m.start() for m in re.finditer(item, context_with_no_punct)]
            count = len(indexes)
            if max < count:
                max = count
    subject_in_context_count.append(max)
    

In [378]:
subject_in_context_count

[0,
 0,
 5,
 0,
 0,
 1,
 4,
 0,
 1,
 1,
 5,
 0,
 5,
 0,
 0,
 5,
 1,
 0,
 5,
 0,
 5,
 5,
 0,
 0,
 0,
 5,
 0,
 5,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 2,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 4,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 2,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 0,
 3,
 2,
 0,
 0,
 3,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 3,
 0,
 3,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 3,
 1,
 1,
 1,
 0,
 1,
 0,
 4,
 4,
 4,
 8,
 0,
 2,
 4,
 8,
 4,
 0,
 0,
 2,
 2,
 4,
 2,
 4,
 0,
 0,
 7,
 0,
 1,
 0,
 2,
 0,
 2,
 1,
 1,
 1,
 2,
 0,
 2,
 1,
 0,
 2,
 1,
 2,
 0,
 0,
 5,
 2,
 1,
 0,
 0,
 2,
 1,
 1,
 0,
 0,
 0,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 3,
 0,
 1,
 2,
 1,
 0,
 4,
 0,
 0,
 2,
 0,
 4,
 0,
 0,
 4,


In [379]:
len(subject_in_context_count)

10570

In [380]:
data['subject_in_context_count'] = subject_in_context_count

In [381]:
data['subject_in_context_count'].value_counts()

0     4595
1     2514
2     1330
3      819
4      470
5      291
6      181
7       92
8       74
9       61
12      26
10      26
11      22
14      15
13      13
16       7
19       6
18       6
15       4
17       3
33       2
38       2
24       2
27       1
35       1
41       1
20       1
25       1
69       1
32       1
23       1
55       1
Name: subject_in_context_count, dtype: int64

In [382]:
# data.to_json('enhanced_'+validation_dataset, orient='records')

### Extract position of answer regarding question subject

In [383]:
# data = pd.read_json('enhanced_'+validation_dataset)

In [384]:
nlp = spacy.load('en_core_web_sm')

In [385]:
def doc_pieces(doc):
    subjects = []
    for ent in doc:
        if ent.dep_ == 'nsubj':
            subjects.append(ent.text)
    return subjects

In [386]:
import string
import re

def extract_answer_position_with_respect_to_subject(data):
    q_subjects = []

    for item in data['question']:
        question = nlp(item)
        q_subjects.append(doc_pieces(question))

    positions = []

    for context, q_sub, answer in zip(data['context'], q_subjects, data['answers']):
        pos = 0
        max = 0
        for item in q_sub:
            if item in context:
                indexes = [m.start() for m in re.finditer(item, context)]
                counter = 0
                for index in indexes:
                    if answer['answer_start'][0] < index:
                        break
                    else:
                        counter += 1
                if max < counter:
                    max = counter
                pos = max
            else:
                pos = -1
        positions.append(pos)
    
    return positions

In [387]:
data['answer_subject_positions'] = extract_answer_position_with_respect_to_subject(data)

In [388]:
data['answer_sunjects'] = q_subjects

In [389]:
data['answer_subject_positions'].value_counts()

 0     3049
-1     2995
 1     2854
 2      888
 3      381
 4      176
 5       80
 6       39
 8       28
 7       26
 9       17
 10      10
 11       7
 13       7
 12       5
 14       3
 15       2
 37       1
 20       1
 26       1
Name: answer_subject_positions, dtype: int64

In [390]:
data['answer_subject_positions'].value_counts()

 0     3049
-1     2995
 1     2854
 2      888
 3      381
 4      176
 5       80
 6       39
 8       28
 7       26
 9       17
 10      10
 11       7
 13       7
 12       5
 14       3
 15       2
 37       1
 20       1
 26       1
Name: answer_subject_positions, dtype: int64

In [391]:
len(data[data.answer_subject_positions >= 3])

784

In [392]:
data.to_json(file_folder+'enhanced_'+validation_dataset, orient='records')